In [ ]:
import sys 
sys.path.append("../")

import nerf_model
import dataloader
import nerf_helpers

import plotly
import torch 
import cv2
from PIL import Image
import itertools 
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
import nerf_helpers

In [ ]:
def visualize(coords, rgb): 
    a, b = coords.shape
    if b == 3:
        coords = coords.T
    if type(rgb) != str:
        rgb = rgb.T
    plot_fig = go.Scatter3d(x=coords[0], y=coords[1], z=coords[2], 
    mode='markers', marker=dict(
       size=2,
       color=rgb
    ))
    return plot_fig

In [ ]:
from importlib import reload 
nerf_helpers = reload(nerf_helpers)

In [ ]:
base_dir = '../tests/test_data/'

ds = dataloader.SyntheticDataset(base_dir, 'train', 4096)
batch = ds[0]
origin = batch['origin']
direc = batch['direc']
rgb = batch['rgba']

dl = dataloader.getSyntheticDataloader(base_dir, 'train', 4096, num_workers=1, shuffle=True)
dl_batch = next(iter(dl))

## Inverse Transform Sampling

In [ ]:
from importlib import reload 
nerf_helpers = reload(nerf_helpers)
import gc 
gc.collect()

In [ ]:
samples, ts = nerf_helpers.generate_coarse_samples(origin, direc, 10)
deltas = nerf_helpers.generate_deltas(ts)

density = torch.rand((4096, 10, 1))
# weights = nerf_helpers.calculate_unnormalized_weights(density, deltas)
weights = torch.rand((4096,10,1))
weights[:,8:,:] += 10
fine_samples, fine_ts = nerf_helpers.inverse_transform_sampling(origin, direc, weights, ts, 100)

print("ts:",ts.shape)
print("weights:", weights.shape)
print("samples:", samples.shape)
print("fine_samples", fine_samples.shape)
print("fine_ts", fine_ts.shape)


In [ ]:
fig = px.histogram(weights.view((-1,1)), nbins=10) #, range_x=(0,1))
fig.show()


In [ ]:
fig = px.histogram(fine_ts.view((-1,1)), nbins=10) #, range_x=(0,1))
fig.show()

fig = make_subplots(specs=[[{"secondary_y": True}]])
pic = visualize(fine_samples.view((-1,3))[::101,:], 'pink')
fig.add_trace(pic)
fig.show()

In [ ]:
fig = px.histogram(ts.view((-1,1)), nbins=10) #, range_x=(0,1))
fig.show()

fig = make_subplots(specs=[[{"secondary_y": True}]])
pic = visualize(samples.view((-1,3)), 'pink')
fig.add_trace(pic)
fig.show()

## Categorial not fun

In [ ]:
from torch.distributions import Categorical

weights = weights / torch.sum(weights, dim=1, keepdims=True)
weights = weights.squeeze(-1)
# weights = torch.ones((4096,64)) / (4096 * 64)
dist = Categorical(weights)
samples = dist.sample((1, 192)).T
fine_ts = torch.gather(ts, 1, samples)
print(samples.shape)
print(fine_ts.shape)

In [ ]:
fig = px.histogram(fine_ts.view((-1,1)))
fig.show()

## Implementing estimate_expected_ray_color

In [ ]:
import nerf_helpers
nerf_helpers = reload(nerf_helpers)
samples = 64
N = 4096
# density: [N x samples x 1]
# rgb: [N x samples x 3]
# deltas: [N x samples x 1]

density = torch.rand((N,samples,1))
rgb = torch.rand((N,samples,3))
deltas = torch.rand((N,samples,1))

neg_delta_density = - 1 * density * deltas

# [N x samples x 3]
transparency =  torch.exp(torch.cumsum(- 1 * neg_delta_density, dim=1))
print(transparency.shape)
weights = 1 - torch.exp(neg_delta_density)
ray_color = torch.sum(transparency * weights * rgb, dim=1)
print(weights.shape)
print(ray_color.shape)

## Implementing Sampling Scheme

In [ ]:
import nerf_helpers
nerf_helpers = reload(nerf_helpers)

print(dl_batch['origin'].shape)
nerf_helpers.fix_batchify(dl_batch)
print(dl_batch['origin'].shape)

coarse = 64
t = torch.linspace(0,1,steps=coarse).unsqueeze(0)
print(t.shape)

In [ ]:
origin = dl_batch['origin'].unsqueeze(1)
direc = dl_batch['direc'].unsqueeze(1)
ts, _ = torch.meshgrid(torch.arange(coarse), torch.arange(4096), indexing='xy')
rand = torch.rand(ts.shape)
ts = (ts + rand) / coarse
ts = ts.unsqueeze(-1)
samples = direc * ts + origin
del _

In [ ]:
origin = dl_batch['origin']
direc = dl_batch['direc']
def generate_samples(o_rays, d_rays, num_samples): 
    N, _ = o_rays.shape
    o_rays = o_rays.unsqueeze(1)
    d_rays = d_rays.unsqueeze(1)
    ts, _ = torch.meshgrid(torch.arange(num_samples), torch.arange(N), indexing='xy')
    del _ 
    rand = torch.rand(ts.shape)
    ts = (ts + rand) / num_samples
    ts = ts.unsqueeze(-1)
    samples = d_rays * ts + o_rays
    return samples, ts

samples, ts = generate_samples(origin, direc, 64)
print(samples.shape)
print(ts.shape)

In [ ]:
def generate_deltas(ts): 
    N, samples, _ = ts.shape
    upper_bound = torch.cat([ts[:,1:,:], torch.ones((N, 1, 1))], dim=1)
    deltas = upper_bound - ts
    return deltas
deltas = generate_deltas(ts)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

samples = samples.view((-1,3))
origin_fig = visualize(samples.T, 'black')
fig.add_trace(origin_fig)

fig.show()

## Check Dataloader Coordinates

In [ ]:
print(rgb.shape)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

origin_fig = visualize(origin.T, rgb)
fig.add_trace(origin_fig)

direc_fig = visualize(direc.T, rgb)
fig.add_trace(direc_fig)


fig.show()

## Generate Test Image

In [ ]:
size = 50
gradient = np.expand_dims(np.linspace([0]*50, [1]*50, num=size, endpoint=True), -1)
image = np.zeros((size, size,3)) + gradient
image[:,:,0] = 1 - image[:,:,0] 
image = (image * 255).clip(0, 255).astype(np.uint8)
# image[:,:,3] = 1 - image[:,:,3] 
print(image.shape)

In [ ]:
im = Image.fromarray(image)
im.save('test1.png')

In [ ]:
def sample_random_coordinates(N, height, width): 
    """Returns [Nx4] randomly sampled coordinates in camera frame
    """
    xs = torch.randint(0, height, size=(N,))
    ys = torch.randint(0, width, size=(N,))
    return xs, ys

torch.manual_seed(0)
xs, ys = sample_random_coordinates(1, 50, 50)
print(xs, ys)
